In [1]:
import torch
import cv2
import numpy as np

def predict_mask(model, image, device):
    """
    Runs inference on a new image and returns the predicted segmentation mask.
    """
    model.eval()
    image = image.to(device).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)  # Model prediction

    predicted_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()  # Convert to NumPy
    return predicted_mask


In [2]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

# 🔹 Load the Model with its original structure
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.DeepLabV3Plus(
    encoder_name="efficientnet-b4",
    encoder_weights=None,  # We are loading our trained weights
    in_channels=3,
    classes=2
)

# 🔹 Load weights (no modifications yet)
checkpoint = torch.load(r"C:\Users\gnvca\OneDrive\Desktop\JP\best_model_effnet_deepl_epoch0.pth", map_location=device)
model.load_state_dict(checkpoint, strict=False)

# 🔹 Now modify the segmentation head AFTER loading weights
model.segmentation_head = nn.Sequential(
    nn.Dropout(0.3),  # 30% dropout
    model.segmentation_head  # Keep the original segmentation head
)

# Move model to device
model.to(device)
model.eval()

print("✅ Model successfully loaded with modified segmentation head!")


C:\Users\gnvca\AppData\Local\Temp\ipykernel_64408\4194772374.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(r"C:\Users\gnvca\OneDrive\Desktop\J

✅ Model successfully loaded with modified segmentation head!


In [ ]:
import torch
import cv2
import numpy as np
import os
import pandas as pd
from torchvision import transforms
from PIL import Image
import segmentation_models_pytorch as smp

# 🔹 Define Paths
image_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\images"
test_csv = r"C:\Users\gnvca\OneDrive\Desktop\JP\Test.csv"
output_dir = r"C:\Users\gnvca\OneDrive\Desktop\JP\output_images"

# 🔹 Image Preprocessing (Matches Training Pipeline)
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize to match model input
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# 🔹 Prediction Function
def predict_mask(model, image, device, original_size):
    image = image.to(device).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)  # Model prediction

    predicted_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()  # Convert to NumPy

    # 🔹 Resize mask back to original image size
    predicted_mask = cv2.resize(predicted_mask, (original_size[1], original_size[0]), interpolation=cv2.INTER_NEAREST)

    return predicted_mask

# 🔹 Read Test Image IDs
df_test = pd.read_csv(test_csv)

# 🔹 Process Each Image in Test.csv
for index, row in df_test.iterrows():
    image_id = row["ID"]  # Extract image ID
    
    image_path = os.path.join(image_dir, f"{image_id}.jpg")
    if not os.path.exists(image_path):
        print(f"❌ Image {image_id}.jpg not found, skipping.")
        continue

    # Load and preprocess image
    original_image = Image.open(image_path).convert("RGB")
    original_size = original_image.size[::-1]  # (height, width)
    
    image = transform(original_image)  # Apply transformations

    # Get Predictions
    predicted_mask = predict_mask(model, image, device, original_size)

    # Convert to BGR image format for OpenCV
    original_image = np.array(original_image)

    # Create an RGB mask overlay
    overlay = np.zeros_like(original_image, dtype=np.uint8)

    # Apply colors based on predicted mask
    overlay[predicted_mask == 0] = (0, 165, 255)  # Orange (Boiler)
    overlay[predicted_mask == 1] = (0, 255, 255)  # Yellow (Solar Panel)

    # Create a boolean mask for areas with predicted labels
    mask_area = predicted_mask > 0  # Only blend where mask is non-zero

    # Expand mask_area to shape (H, W, 1) for broadcasting
    mask_area_3ch = np.stack([mask_area]*3, axis=-1)  # shape: (H, W, 3)

    # Blend full image
    blended_full = cv2.addWeighted(original_image, 0.6, overlay, 0.4, 0)

    # Create mask for where to blend
    mask_area = predicted_mask > 0
    mask_area_3ch = np.repeat(mask_area[:, :, np.newaxis], 3, axis=2)

    contours, _ = cv2.findContours((predicted_mask == 1).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(original_image, contours, -1, (0, 255, 255), thickness=2)

    # Apply blend only where mask is active
    blended_image = np.where(mask_area_3ch, blended_full, original_image)

    # Save raw mask as grayscale image
    mask_output_path = os.path.join(output_dir, f"{image_id}_mask.jpg")
    cv2.imwrite(mask_output_path, (predicted_mask * 127).astype(np.uint8))  # Scale class 0→0, 1→127 for visibility

    # Save the resulting image
    output_path = os.path.join(output_dir, f"{image_id}_pred.png")
    cv2.imwrite(output_path, cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR))

    print(f"✅ Saved: {output_path}")

print(f"\n🎉 Processing complete! All predicted images are saved in `{output_dir}`.")
